# Spracovanie prirodzeného jazyka (zatiaľ bez DL)

Veľké množstvo textových dát môže slúžiť ako zdroj cenných informácií a práve preto sa v posledných rokoch **spracovanie prirodzeného jazyka** (*natural language processing* - NLP) stalo dôležitou podoblasťou strojového učenia. Na nasledujúcich cvičeniach sa pozrieme na základné princípy NLP a úlohy špecifické pre texty. Na dnešnom cvičení si ukážeme príklad takzvanej analýzy sentimentu, teda kategorizácie textu podľa prístupu a názoru autora.

Dnes zatiaľ nebudeme na to používať neurónové siete, keďže je dôležité, aby ste pochopili postupnosť krokov pri spracovaní textu. Na ďalšom cvičení si ukážeme, ako špeciálny typ neurónových sietí, rekurentná NS, pomôže pri tejto úlohe. Náš výlet v oblasti NLP uzavrieme pohľadom na attention mechanizmy a transformery ako State of the Art pre NLP.

Na dnešnom cvičení budeme pracovať s datasetom IMDb, ktorý obsahuje 50 000 recenzií filmov, ktoré sú zaradené do dvoch kategórií: pozitívnych (viac ako 6 hviezdičiek) a negatívnych (menej ako 5 hviezdičiek). Po stiahnutí datasetu nás čakajú úlohy ako:

1. predspracovanie dát;
2. vektorizácia textových údajov;
3. trénovanie modelu strojového učenia pre klasifikáciu;
4. práca s veľkými textovými dátami;
5. estimácia obsahu textu.

## 1. Predspracovanie dát

[Dataset si môžete stiahnuť z tohto odkazu](http://ai.stanford.edu/~amaas/data/sentiment/), následne si rozbaľte súbor (cca 80 MB).

Môžete si všimnúť, že dáta sú rozdelené do dvoch adresárov na trénovanie a testovanie, a v rámci týchto priečinkov nájdeme veľa súborov. Pre pohodlnejšiu prácu si tieto dáta nakopírujeme do jedného CSV súboru (proces môže trvať niekoľko minút). Ak sa vám nechce čakať na výsledky, [môžete si stiahnuť hotový CSV súbor](lab07/movie_data.csv) (cca 64 MB).

Rozbalenie môžete urobiť priamo v Pythone, čo by mohlo byť rýchlejšie:

In [ ]:
import tarfile
with tarfile.open("lab07/aclImdb_v1.tar.gz", 'r:gz') as tar:
    tar.extractall()

In [ ]:
import os
import sys

import pandas as pd
# pip install pyprind
import pyprind

BASEPATH = "lab07/aclImdb"

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for subdir in ('test', 'train'):
    for cat in ('pos', 'neg'):
        path = os.path.join(BASEPATH, subdir, cat)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[cat]]], ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

Pred ukladaním načítaných dát si ich pomiešame v náhodnom poradí, a následne ich uložíme do CSV súboru (ak ste si stiahli hotový súbor, tieto kroky môžete vynechať).

In [ ]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv("movie_data.csv", index=False, encoding='utf-8')

Následne si dataset znova načítame (tento krok už potrebujete urobiť), trošku upravíme, a skontrolujeme množstvo a obsah dát:

In [ ]:
df = pd.read_csv("movie_data.csv", encoding='utf-8')
# krok potrebny na niektorych pocitacoch
# df = df.rename(columns={"0": "review", "1": "sentiment"})
print(df.shape)
print(df.head(3))

## 2. Vektorizácia textových údajov

Neurónové siete, ale aj ďalšie algoritmy strojového učenia boli navrhnuté tak, aby dokázali pracovať s číselnými dátami, čo sa nedá povedať o texte. Práve preto v prípade textu a iných kategorických údajov je potrebné tieto údaje pretransformovať do číselnej reprezentácie. V tomto kroku použijeme prístup **bag-of-words**, ktorý každému slovu pridelí jedinečný príznakový vektor. Tento proces sa uskutoční v dvoch krokoch:

1. vytvoríme si zásobu jedinečných tokenov - napríklad slov - zo všetkých dokumentov.
2. zostrojíme príznakový vektor z každého dokumentu, kde vektor obsahuje informáciu o tom, koľkokrát sa dané slovo vyskytuje v danom dokumente.

Je jasné, že väčšina hodnôt vo vektoroch bude 0, t.j. vektory budú **sparse**, čo je presne to, čo potrebujeme.

### 2.1. Generovanie príznakových vektorov

Pre generovanie vektorov použijeme knižnicu `scikit-learn`, ktorá je súčasťou inštalácie Anaconda. Proces si ukážeme na jednoduchých dátach, a neskôr ho aplikujeme na náš dataset.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['Roses are red',
                 'Violets are blue',
                 'Roses are read, violets are blue, wine costs less than dinner for two'])
bag = count.fit_transform(docs)

Následne si môžeme skontrolovať a analyzovať obsah vygenerovaných vektorov:

In [ ]:
print(count.vocabulary_)

In [ ]:
print(bag.toarray())

Vygenerovaná zásoba slov reprezentuje index daného slova vo vektorovej reprezentácii a vektory obsahujú počet výskytov daného slova vo vete. Tento počet nazývame aj ako **raw term frequencies**: *tf(t, d)* - počet výskytov výrazu *t* v dokumente *d*. Na poradí týchto výrazov nám nezáleží, ich poradie je odvodené z indexov zásoby slov (zvyčajne podľa abecedy).

**Poznámka**: V našom bag-of-words modeli sme použili 1-gram (unigram) model, ale existujú aj iné reprezentácie, kde sa jeden výraz skladá z viacerých tokenov, napríklad bigram: *roses are*, *are red*. Rôzne úlohy vyžadujú rôznu aritu reprezentácie.

Nevýhoda TF reprezentácie je, že sa niektoré slová často vyskytujú v príkladoch oboch typov (pozitívne a negatívne), a práve preto zvyčajne nemajú veľkú výpovednú hodnotu pre klasifikáciu. Namiesto toho teda, aby sme brali do úvahy ich surovú početnosť v dátach, môžeme použiť techniku **term frequency-inverse document frequency**: $$tf{\text -}idf(t, d) = tf(t, d) \times idf(t, d),$$

kde

$$
idf(t,d) = log\frac{n_{d}}{1 + df(d, t)}.
$$

$n_{d}$ je celkový počet dokumentov, a $df(d, t)$ je počet dokumentov *d*, ktoré obsahujú výraz *t*. Konkrétne implementácie v knižnici `scikit-learn` fungujú s menšími zmenami, ale to nás nemusí zaujímať.

Našu reprezentáciu vieme prekonvertovať do TF-IDF formy pomocou kódu:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

### 2.2 Čistenie dát

Skutočné textové dáta často obsahujú špeciálne znaky, ktoré nemajú žiadnu výpovednú hodnotu, a práve preto by bolo vhodné ich vymazať. Zoberte príklad z jednej recenzie:

In [ ]:
print(df.loc[0, 'review'][-50:])

Text obsahuje HTML markupy, ako aj interpunkčné znamienka. V niektorých prípadoch síce interpunkcia zohráva veľkú rolu pri vyhodnocovaní textu, v našom prípade ju však nepotrebujeme, takže odstránime ju spolu s HTML tagmi. K tomu ešte spracujeme aj smajlíky ako bonus.

In [ ]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [ ]:
print(preprocessor(df.loc[0, 'review'][-50:]))
print(preprocessor("</a>This :) is :( a test :-)!"))

Predspracovanie teraz už vieme aplikovať na našich skutočných dátach:

In [ ]:
df['review'] = df['review'].apply(preprocessor)

### 2.3. Tokenizácia dokumentov

Prvý krok pri spracovaní textu je jeho rozdelenie na menšie bloky, tzv. tokeny, ktoré väčšinou sú slová. Základným spôsobom rozdelenia do slov je rozdelenie viet podľa medzier:

In [ ]:
def tokenizer(text):
    return text.split()

print(tokenizer("runners like running and thus they run"))

### 2.4. Stemming (a lematizácia)

Ako môžete vidieť na príklade vyššie, niektoré slová (ako *running* a *run*, sčasti aj *runners*) reprezentujú podobný koncept, práve preto je zbytočné ich mať viackrát v zásobe slov. Takisto by sme potrebovali spracovať množné čísla a previesť slová do singulárneho tvaru. Tento proces sa nazýva **stemming** a existuje niekoľko algoritmov na jeho realizáciu, jeden zo základných je **Porter stemming**, ktorý je dostupný v knižnici Natural Language Toolkit (súčasťou Anacondy):

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter("runners like running and thus they run"))

Na vyššom príklade vidíte efekt stemmingu, ktorý ale nie je dokonalý proces (napríklad *thus* nesprávne zjednodušil na tvar *thu*). Druhý podobný proces - lematizácia - vám vždy vráti základný slovníkový tvar slova, je však výpočtovo náročnejší aj keď zvyčajne dáva lepšie výsledky.

### 2.5. Odstránenie stop slov

Stop slová sú slová, ktoré sa často vyskytujú vo všetkých textoch a práve preto ich nemá veľký zmysel spracovať, keďže nemajú veľkú informačnú hodnotu pre klasifikáciu alebo inú úlohu strojového učenia. Síce *tf-idf* čiastočne eliminuje potrebu odstránenia takýchto slov, keďže zmenší dôležitosť často sa opakujúcich slov, v niektorých prípadoch je dôležité tieto slová odstrániť, na čo existujú hotové množiny pre rôzne jazyky.

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
print([w for w in tokenizer_porter("a runner likes running and runs a lot") if w not in stop])

## 3. Trénovanie klasifikačného modelu

Na dnešnom cvičení síce ešte nebudeme používať neurónové siete, proces trénovania klasifikačného modelu si však ukážeme pomocou logistickej regresie. Pri trénovaní použijeme aj optimalizáciu hyperparametrov pomocou `sklearn`. Najprv si ale pripravíme trénovacie a testovacie dáta:

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

Následne si natrénujeme model:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
small_param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [None],
        'vect__tokenizer': [tokenizer, tokenizer_porter],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    },
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l2'],
        'clf__C': [1.0, 10.0]
    }
]
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy', cv=5,
                           verbose=2, n_jobs=1)  # n_jobs=-1 for parallel processing
gs_lr_tfidf.fit(X_train, y_train)

Najúspešnejšie nastavenie parametrov a príslušné výsledky nájdeme pomocou kódu:

In [ ]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Môžeme to overiť aj na testovacích dátach:

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

## 4. Práca s veľkými textovými dátami

V predošlom kroku ste si mohli všimnúť, že optimalizácia hyperparametrov môže trvať pomerne dlho pri predspracovaní údajov ako tokenizácia a následný stemming. Ak máte veľký dataset, kľudne sa môže stať, že sa vám celý dataset ani nezmestí do pamäti počítača, čo spôsobí zlyhanie takéhoto vyhľadávania.

Podobné problémy pri neurónových sieťach sa riešia minibatch trénovaním, a obdobný prístup existuje aj pre iné modely, tzv. **out-of-core learning**, kde model trénujeme iba parciálne na menšom počte príkladov pomocou funkcie `partial_fit`.

V prvom kroku si zadefinujeme nový tokenizer s odstránením stop slov:

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
def new_tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

Ďalej si zadefinujeme generátor, ktorý nám vráti jednotlivé dokumenty z celkového datasetu:

In [ ]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [ ]:
next(stream_docs(path="movie_data.csv"))

Ďalšia funkcia nám dodá jeden minibatch trénovacích údajov:

In [ ]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

Pri vektorizácii teraz už nemôžeme používať `CountVectorizer` alebo `TfidfVectorizer`, keďže tieto metódy vyžadujú informáciu o celkovom počte výskytov jednotlivých slov v datasete. Namiesto toho použijeme iný typ vektorizácie, konkrétne `HashingVectorizer`, ktorý je nezávislý od dát:

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,  # vector size - estimate
                         preprocessor=None,
                         tokenizer=new_tokenizer)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path="movie_data.csv")

Konečne môžeme začať aj trénovanie:

In [ ]:
import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

Po úspešnom trénovaní môžeme skontrolovať aj trénovaciu presnosť:

In [ ]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print(f'Accuracy: {clf.score(X_test, y_test):.3f}')

Finálna presnosť môže byť trošku nižšia ako v predošlom prípade, avšak out-of-core trénovanie je omnoho rýchlejšie a menej zaťažuje pamäť počítača.

Keďže "testovaciu" množinu sme použili na validáciu, ak sme už spokojní s výsledkami, môžeme tieto dáta použiť aj na trénovanie:

In [ ]:
clf = clf.partial_fit(X_test, y_test)

## 5. Estimácia obsahu textu

Estimácia obsahu textu nám môže pomôcť pri zhlukovaní neolabelovaných textových dát, kde sa snažíme prideliť lable jednotlivým textom na základe ich obsahu. Jedným z možných algoritmov na riešenie takéhoto problému je **latent Dirichlet allocation**, alebo LDA, ktorá je založená na Bayesovej inferencii. Jej cieľom je nájsť slová, ktoré sa často vyskytujú spoločne vo viacerých dokumentoch, a tak definujú tému, resp. kategóriu. Jej vstupom je bag-of-words model, z ktorého vygeneruje dve matice: document-to-topic a word-to-topic, pričom ich znásobením dostaneme naspäť pôvodný text s čo najmenšou chybou. Hyperparametrom LDA je počet hľadaných kategórií.

In [ ]:
import pandas as pd

df = pd.read_csv("movie_data.csv", encoding='utf-8')
# krok potrebny na niektorych pocitacoch
# df = df.rename(columns={"0": "review", "1": "sentiment"})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,  # ignore words with a frequency above 10%
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)
lda.components_.shape

Po trénovaní môžeme vizualizovať najčastejšie slová v jednotlivých kategóriách:

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                    [:-n_top_words - 1:-1]]))

Ukážkové príklady pre kategóriu *horror*:

In [ ]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f'\nHorror movie #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')

## Použité zdroje

* **IMDb dataset**: Maas, Andrew, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. "Learning word vectors for sentiment analysis." In Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies, pp. 142-150. 2011.
* **Porter stemmer:** Porter, Martin F. "An algorithm for suffix stripping." Program 14, no. 3 (1980): 130-137.
* **Natural Language Toolkit:** https://www.nltk.org
* **Latent Dirichlet allocation:** Blei, David M., Andrew Y. Ng, and Michael I. Jordan. "Latent dirichlet allocation." Journal of machine Learning research 3, no. Jan (2003): 993-1022.
* Raschka, Sebastian, Yuxi Hayden Liu, Vahid Mirjalili, and Dmytro Dzhulgakov. Machine Learning with PyTorch and Scikit-Learn: Develop machine learning and deep learning models with Python. Packt Publishing Ltd, 2022.